In [1]:
import hist
import cloudpickle
import zlib
import numpy as np

In [2]:
pathResults = "/gwdata/users/gpizzati/condor_processor/results"
with open(f"{pathResults}/results_merged.pkl", "rb") as file:
    results = cloudpickle.loads(zlib.decompress(file.read()))

results = results["results"]


xss = {
    "Zjj": 2.78,
    "DY": 6077.22,
    "DY_inc": 6077.22,
    "DY_hard": 6077.22,
    "DY_PU": 6077.22,
    "TTTo2L2Nu": 87.310,
    "ST_t-channel_top": 44.07048,
    "ST_t-channel_antitop": 26.2278,
    "ST_tW_antitop": 35.85,
    "ST_tW_top": 35.85,
    "ST_s-channel": 3.34368,
}

datasets = {
    "Data": {
        "samples": [
            "DoubleMuon",
            "SingleMuon",
            "EGamma",
            # "MuonEG",
        ],
        "is_data": True,
    },
    "Top": {
        "samples": [
            "TTTo2L2Nu",
            "ST_s-channel",
            "ST_t-channel_top",
            "ST_t-channel_antitop",
            "ST_tW_antitop",
            "ST_tW_top",
        ],
    },
    "DY": {
        "samples": ["DY_inc"],
    },
    # "DY_PU": {
    #     "samples": ["DY_PU"],
    # },
    # "DY_hard": {
    #     "samples": ["DY_hard"],
    # },
    "Zjj": {
        "samples": ["Zjj"],
    },
}

lumi = 7066.552169 / 1000


def renorm(h, xs, sumw):
    scale = xs * 1000 * lumi / sumw
    # print(scale)
    _h = h.copy()
    a = _h.view(True)
    a.value = a.value * scale
    a.variance = a.variance * scale * scale
    return _h


def fold(h):
    # fold first axis
    # print(h.shape)
    _h = h.copy()
    a = _h.view(True)

    a.value[1, :] = a.value[1, :] + a.value[0, :]
    a.value[0, :] = 0

    a.value[-2, :] = a.value[-2, :] + a.value[-1, :]
    a.value[-1, :] = 0

    a.variance[1, :] = a.variance[1, :] + a.variance[0, :]
    a.variance[0, :] = 0

    a.variance[-2, :] = a.variance[-2, :] + a.variance[-1, :]
    a.variance[-1, :] = 0

    # a.value[1] = a.value[1] + a.value[0]
    # a.value[-2] = a.value[-2] + a.value[-1]
    # a.value[0] = 0
    # a.value[-1] = 0
    return _h


def get_variations(h):
    axis = h.axes[1]
    variation_names = [axis.value(i) for i in range(len(axis.centers))]
    return variation_names


region = "sr_mm"
variable = "mll"
regions = ["top_cr", "vv_cr", "sr"]
#regions = ["sr_inc", "sr_0j", "sr_1j", "sr_2j", "sr_geq_2j"]
regions = ["sr_inc"]
# regions = ["sr_geq_2j"]
categories = ["ee", "mm"]
regions = [f"{region}_{category}" for region in regions for category in categories]
variables = [
    "ptll",
]

print("Start converting histograms")

histos = {}
for region in regions:
    histos[region] = {}
    for variable in variables:
        _histos = {}
        axis = 0
        for histoName in datasets:
            for sample in datasets[histoName]["samples"]:
                if sample not in results:
                    print("Skipping", sample)
                    continue

                h = results[sample]["h"][variable][:, hist.loc(region), :].copy()

                # renorm mcs
                if not datasets[histoName].get("is_data", False):
                    h = renorm(h, xss[sample], results[sample]["sumw"])
                h = fold(h)

                # if histoName == 'Zjj':
                #     print(np.sum(h[:, hist.loc('nom')].values(True)))

                if isinstance(axis, int):
                    axis = h.axes[0]  # .copy()
                histo = {}
                variation_names = get_variations(h)
                # for variation_name in variation_names:
                #     if variation_name == "nom":
                #         continue
                #     histo[variation_name] = h[:, hist.loc(variation_name)].values()

                nom = h[:, hist.loc("nom")].values()
                histo["nom"] = nom

                # stat = np.sqrt(h[:, hist.loc("nom")].variances())
                # histo["stat_up"] = nom + stat
                # histo["stat_down"] = nom - stat

                if histoName not in _histos:
                    _histos[histoName] = {}  # "nominal": nom.copy()}
                    for vname in histo:
                        _histos[histoName][vname] = histo[vname]
                else:
                    for vname in histo:
                        _histos[histoName][vname] += histo[vname]
        histos[region][variable] = {"histos": _histos, "axis": axis}

print("Done converting histograms")


Start converting histograms
Done converting histograms


In [3]:
histos

{'sr_inc_ee': {'ptll': {'histos': {'Data': {'nom': array([430877., 540529., 376003., 258010., 184083., 135796., 103419.,
             80514.,  64129.,  51943.,  42265.,  35122.,  29063.,  24342.,
             20157.,  17178.,  14773.,  12504.,  10570.,   9152.,   7953.,
              6894.,   5930.,   5091.,   4479.,   3808.,   3488.,   3062.,
              2703.,  24487.])},
    'Top': {'nom': array([ 23.54152272,  69.86822552, 115.76872189, 153.9797687 ,
            187.92420613, 214.60770165, 238.84184729, 260.20399163,
            276.16262007, 291.53249631, 294.09402594, 295.66581799,
            297.89070656, 286.83440976, 272.62205334, 259.71713967,
            235.52650267, 211.94903083, 189.42482988, 166.17790212,
            143.94410986, 124.93024768, 106.3196831 ,  88.52814207,
             74.77625192,  60.31451066,  50.44185197,  39.5839984 ,
             34.60069699, 165.93278893])},
    'DY': {'nom': array([527690.33653178, 527960.66263938, 334595.5814734 , 234588.60162

In [15]:
region = 'sr_inc_mm'
h_mm = histos[region]['ptll']['histos']['Data']['nom'].copy()
for histo in ['Top', 'Zjj']:
    h_mm = h_mm - histos[region]['ptll']['histos'][histo]['nom'].copy()
correction = h_mm/histos[region]['ptll']['histos']['DY']['nom']
edges = results['DY_inc']['h']['ptll'][:,hist.loc(region),hist.loc('nom')].axes[0].edges
h = hist.Hist(hist.axis.Variable(edges,name='ptll'),hist.storage.Double())

In [14]:
a= h.view(True)
a.value[:]= correction

ValueError: could not broadcast input array from shape (30,) into shape (32,34)

In [10]:
results['DY_inc'].keys()

dict_keys(['sumw', 'nevents', 'h'])